In [3]:
from astropy.io import fits
import numpy as np
import astropy
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats 
from scipy.optimize import curve_fit

from matplotlib.backends.backend_pdf import PdfPages

In [107]:
file_path = '/home/arya/Desktop/30K_DATA/det0_good_data_30k/fits_file0/20230706_1006_ba133_30000pkts_ext_hv_600_20kev_spectra.fits'
hdulist = fits.open(file_path)

In [109]:
data = hdulist[1].data
print(Table(data))

   time   detid pixid pha 
--------- ----- ----- ----
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
        0     0     0    0
      ...   ...   ...  ...
535212659     0    40  381
535213560     0    43 2620
535213562     1    78 2928
535214179     0    38 1289
535215298     0    38 1230
535215391     1   103 1065
535219701     0    55  364
535219744     1   106  497
535219754     0    41  897
535219795     1   105 4095
Length = 3842560 rows


In [111]:
# Filter data for detid == 
detid = data[data['detid'] == int(input("Detector Id (0/1) = "))]

Detector Id (0/1) =  0


In [113]:
source = input("source name (Am/Ba1/Ba2/Eu1/Eu2) = ")
if source == "Am":
    pha_min = 816  # Minimum PHA value
    pha_max = 2000 # Maximum PHA value
    Energy_value = 59.6
    bin_count = 24
elif source == "Ba1":
    pha_min = 304 
    pha_max = 672 
    Energy_value = 30.85
    bin_count = 23
elif source == "Ba2":
    pha_min = 1120  
    pha_max = 1616 
    Energy_value = 81
    bin_count = 31
elif source == "Eu2":
    pha_min = 1632  
    pha_max = 2000
    Energy_value = 105.31
    bin_count = 23
else: 
    pha_min = 1200
    pha_max =1760
    Energy_value = 85.55
    bin_count = 35
print(pha_min,pha_max,Energy_value)

source name (Am/Ba1/Ba2/Eu1/Eu2) =  Ba2


1100 1700 81


In [115]:
n = len(detid)
k = 1 + np.log2(n)
bin_count = 64  # int(4096 / k)
max_count_pha_list = []
max_count_pha_list_OFF = []
def gauss(x,amp,mean,stdev):
    return amp*np.exp(-(x-mean)**2/(2*stdev**2))

# Loop through pixid values from 0 to 255
for pixid in range(256):
    # Filter data for the current pixid
    data_ = detid[detid['pixid'] == pixid]
    data_ = data_['pha']
    filtered_data = data_[(data_ != 4095) & (data_ >= pha_min) & (data_ <= pha_max)]

    if len(filtered_data) > 10:  # Ensure there is data to process
        # Compute histogram
        N, bins = np.histogram(filtered_data, bins=bin_count)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        
        # Initialize variables for fitting
        p0 = [np.max(N), np.mean(filtered_data), np.std(filtered_data)]
        params = None
        pcov = None
        
        for i in range(10):
            try:
                # Fit the Gaussian function to the histogram data
                params, pcov = curve_fit(gauss, bin_centers, N, p0, maxfev=200)
                
                # Check if the change in parameters is within the acceptable range
                if np.all((np.abs(p0 - params) / np.sqrt(np.diag(pcov))) >1):
                    break
                
                # Update initial guess for next iteration
                p0 = params
            except RuntimeError:
                # If the fit fails, continue to the next iteration
                continue

        if params is not None:
            # Append the mean value (mu) from the fit to the list
            max_count_pha_list.append((pixid, params[1], np.sqrt(np.diag(pcov))[1], Energy_value))
        else:
            # If fit was unsuccessful, append None
            max_count_pha_list_OFF.append((pixid, None, Energy_value))
    else:
        # Append None if there is no data for the current pixid
        max_count_pha_list_OFF.append((pixid, None, Energy_value))

print("List of maximum count PHA values for each pixid:")
print(max_count_pha_list)
print("List of unsuccessful fits for each pixid:")
print(max_count_pha_list_OFF)


List of maximum count PHA values for each pixid:
[(17, 1230.6384651454352, 4.451215458935229, 81), (18, 1257.0414658811092, 4.3612894964459, 81), (19, 1223.1453356337365, 3.0252893477177984, 81), (20, 1278.345589489806, 2.713222628280727, 81), (21, 1292.432429679919, 2.1324978768620158, 81), (22, 1260.755761204315, 1.4994998614919732, 81), (23, 1278.782237864332, 1.6580882491596383, 81), (24, 1331.023967882171, 1.770195522831849, 81), (25, 1333.9085759484046, 1.6715078445986613, 81), (26, 1294.5082310912467, 1.5614216180125329, 81), (27, 1353.0092112963523, 2.2248938971739496, 81), (28, 1313.835566021495, 2.942788587387475, 81), (29, 1329.8661642448621, 3.316369006475081, 81), (30, 1291.8624868840382, 3.8462641436284213, 81), (32, 1272.4812970383907, 9.39450520904598, 81), (33, 1295.7786071926637, 6.070231962645211, 81), (34, 1284.1835991366684, 4.410790947891884, 81), (35, 1235.886091990815, 2.894263568287969, 81), (36, 1273.8612750470188, 2.883029998808814, 81), (37, 1249.15944116606

In [117]:
import csv
import os

# Specify the CSV file path
csv_filename = '/home/arya/Desktop/30K_DATA/det0_good_data_30k/final_fitting_D0.csv'

# Function to check if the file is empty
def is_file_empty(filename):
    return os.path.exists(filename) and os.path.getsize(filename) == 0

# Write results to CSV file
with open(csv_filename, mode='a', newline='') as f:
    writer = csv.writer(f)
    
    # Check if the file is empty and write the header if it is
    if not os.path.exists(csv_filename) or is_file_empty(csv_filename):
        writer.writerow(['pixel no.', 'mean_values', 'errors', 'Energy_values'])
    
    # Write the results
    for max_count_pha_list in max_count_pha_list:
        writer.writerow(max_count_pha_list)

print(f"Results saved to '{csv_filename}'")


Results saved to '/home/arya/Desktop/30K_DATA/det0_good_data_30k/final_fitting_D0.csv'


In [ ]:
import os
# Specify the FITS file path
fits_filename = '/home/arya/Desktop/30K_DATA/det1_good_data_30k/final_fitting_D1.fits'
# Convert the data to a structured numpy array
dtype = [('pixel no.', 'i4'), ('mean_values', 'f4'), ('errors', 'f4'), ('Energy_values', 'f4')]
data_array = np.array(max_count_pha_list, dtype=dtype)

# Create a PrimaryHDU object to encapsulate the data
hdu = fits.BinTableHDU(data=data_array)

# Check if the file exists and remove it if it does
if os.path.exists(fits_filename):
    os.remove(fits_filename)

# Write the data to the FITS file
hdu.writeto(fits_filename)

print(f"Results saved to '{fits_filename}'")
